_한국어로 기계번역됨_


# 스레드 수준의 지속성 추가 방법 (기능적 API)

!!! 정보 "전제 조건"

    이 가이드는 다음에 대한 이해를 전제로 합니다:
    
    - [기능적 API](../../concepts/functional_api/)
    - [지속성](../../concepts/persistence/)
    - [메모리](../../concepts/memory/)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)

많은 AI 애플리케이션은 동일한 [스레드](../../concepts/persistence#threads)에서 여러 상호작용 간에 컨텍스트를 공유하기 위해 메모리가 필요합니다(예: 대화의 여러 턴). LangGraph 기능적 API에서는 이와 같은 메모리를 [스레드 수준의 지속성](https://langchain-ai.github.io/langgraph/concepts/persistence)을 사용하여 모든 [엔트리 포인트](langgraph.func.entrypoint) 워크플로에 추가할 수 있습니다.

LangGraph 워크플로를 생성할 때, [체크포인터](https://langchain-ai.github.io/langgraph/reference/checkpoints/#basecheckpointsaver)를 사용하여 결과를 지속하도록 설정할 수 있습니다:

1. 체크포인터 인스턴스 생성:

    ```python
    from langgraph.checkpoint.memory import MemorySaver
    
    checkpointer = MemorySaver()       
    ```

2. `entrypoint()` 데코레이터에 `checkpointer` 인스턴스 전달:

    ```python
    from langgraph.func import entrypoint
    
    @entrypoint(checkpointer=checkpointer)
    def workflow(inputs):
        ...
    ```

3. 워크플로 함수 시그니처에서 `previous` 매개변수를 선택적으로 노출:

    ```python
    @entrypoint(checkpointer=checkpointer)
    def workflow(
        inputs,
        *,
        # 워크플로 함수 시그니처에 `previous`를 선택적으로 지정하여
        # 마지막 실행 시 워크플로의 반환값에 접근할 수 있습니다.
        previous
    ):
        previous = previous or []
        combined_inputs = previous + inputs
        result = do_something(combined_inputs)
        ...
    ```

4. 워크플로에서 어떤 값을 반환하고 어떤 값을 체크포인터가 `previous`로 저장할지 선택적으로 결정:

    ```python
    @entrypoint(checkpointer=checkpointer)
    def workflow(inputs, *, previous):
        ...
        result = do_something(...)
        return entrypoint.final(value=result, save=combine(inputs, result))
    ```

이 가이드는 워크플로에 스레드 수준의 지속성을 추가하는 방법을 보여줍니다.

!!! 팁 "참고"

    여러 대화나 사용자 간에 __공유__되는 메모리가 필요하다면 (교차 스레드 지속성), 이 [가이드](../cross-thread-persistence-functional)를 확인하세요.

!!! 팁 "참고"

    `StateGraph`에 스레드 수준의 지속성을 추가해야 한다면 이 [가이드](../persistence)를 확인하세요.


## 설정

먼저 필요한 패키지를 설치해야 합니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic


다음으로, 우리가 사용할 LLM인 Anthropic의 API 키를 설정해야 합니다.


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 개선하세요. LangSmith는 추적 데이터를 사용하여 LangGraph로 구축된 LLM 앱의 디버깅, 테스트 및 모니터링을 가능하게 합니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 읽어보세요.
    </p>
</div>


## 예시: 단기 기억이 있는 간단한 챗봇

우리는 단일 작업을 호출하는 워크플로를 사용할 것입니다 [챗 모델](https://python.langchain.com/docs/concepts/chat_models/).

먼저 사용할 모델을 정의해 보겠습니다:


In [3]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")


이제 우리의 작업과 워크플로를 정의할 수 있습니다. 지속성을 추가하기 위해, [entrypoint()][langgraph.func.entrypoint] 데코레이터에 [Checkpointer](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.base.BaseCheckpointSaver)를 전달해야 합니다.


In [4]:
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver


@task
def call_model(messages: list[BaseMessage]):
    response = model.invoke(messages)
    return response


checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def workflow(inputs: list[BaseMessage], *, previous: list[BaseMessage]):
    if previous:
        inputs = add_messages(previous, inputs)

    response = call_model(inputs).result()
    return entrypoint.final(value=response, save=add_messages(inputs, response))


이 워크플로우를 사용하려고 하면 대화의 맥락이 상호작용 간에 유지됩니다.


!!! 참고 사항

    LangGraph Cloud 또는 LangGraph Studio를 사용하고 있다면, 체크포인터를 진입점 데코레이터에 전달할 필요가 없습니다. 이는 자동으로 수행됩니다.


이제 에이전트와 상호 작용할 수 있으며, 이전 메시지를 기억하고 있음을 알 수 있습니다!


In [5]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "hi! I'm bob"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()


================================== Ai Message ==================================

Hi Bob! I'm Claude. Nice to meet you! How are you today?


이전에 진행한 대화를 언제든지 재개할 수 있습니다.


In [6]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()


================================== Ai Message ==================================

Your name is Bob.


새로운 대화를 시작하고 싶다면 다른 `thread_id`를 전달하면 됩니다. 뿅! 모든 기억이 사라집니다!


In [7]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in workflow.stream(
    [input_message],
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
):
    chunk.pretty_print()


================================== Ai Message ==================================

I don't know your name unless you tell me. Each conversation I have starts fresh, so I don't have access to any previous interactions or personal information unless you share it with me.


!!! 팁 "스트리밍 토큰"

    챗봇에서 LLM 토큰을 스트리밍하려면 `stream_mode="messages"`를 사용할 수 있습니다. 자세한 내용을 보려면 이 [방법 안내서](../streaming-tokens)를 확인하세요.
